<a href="https://colab.research.google.com/github/ethemkaral/DataScience/blob/main/RecommenderSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recomender System

In [ ]:
import pandas

#TEST

In [ ]:
!git clone https://github.com/ethemkaral/DataScience.git

Cloning into 'DataScience'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 22 (delta 9), reused 2 (delta 0), pack-reused 0
Receiving objects: 100% (22/22), 78.41 KiB | 1.54 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
#print(check_output(["pip" , "install" ,"requests"]).decode("utf8"))



# Load data
df = pd.DataFrame()


In [ ]:
df = pd.read_csv('/content/DataScience/TopStaredRepositories.csv')
df.set_index(['Repository Name'])

df.head(2)

,Username,Repository Name,Description,Last Update Date,Language,Number of Stars,Tags,Url,Gravatar
0,freeCodeCamp,freeCodeCamp,The https://freeCodeCamp.com open source codeb...,2017-06-24T15:56:17Z,JavaScript,290k,"nonprofits,certification,curriculum,react,node...",https://github.com/freeCodeCamp/freeCodeCamp,https://avatars0.githubusercontent.com/u/98925...
1,twbs,bootstrap,"The most popular HTML, CSS, and JavaScript fra...",2017-06-24T15:40:21Z,JavaScript,112k,"javascript,css,html,bootstrap,jekyll-site,scss",https://github.com/twbs/bootstrap,https://avatars0.githubusercontent.com/u/29185...


In [ ]:
## Cleaning data
# We fill the emptpy URL cells
df['Url'] = "http://github.com/" +         df['Username'] + "/" + df['Repository Name']
# We add a final comma character for the tag string, it will be usefull when we tokenize
df['Tags'].fillna("", inplace=True)
df['Tags'] = df['Tags'] + ","

# We do not want uppercase on any label
df['Language'] = df.loc[:, 'Language'].str.lower()
# Copy a backup variable, so we can change our main dataframe
df_backup = df.copy(deep=True)
df.head(2)

,Username,Repository Name,Description,Last Update Date,Language,Number of Stars,Tags,Url,Gravatar
0,freeCodeCamp,freeCodeCamp,The https://freeCodeCamp.com open source codeb...,2017-06-24T15:56:17Z,javascript,290k,"nonprofits,certification,curriculum,react,node...",http://github.com/freeCodeCamp/freeCodeCamp,https://avatars0.githubusercontent.com/u/98925...
1,twbs,bootstrap,"The most popular HTML, CSS, and JavaScript fra...",2017-06-24T15:40:21Z,javascript,112k,"javascript,css,html,bootstrap,jekyll-site,scss,",http://github.com/twbs/bootstrap,https://avatars0.githubusercontent.com/u/29185...


In [ ]:
# Generate tag list
mergedlist = []
for i in df['Tags'].dropna().str.split(","):
    mergedlist.extend(i)
tags = sorted(set(mergedlist))
# Encode languages in single column
just_dummies = pd.get_dummies(df['Language'])
for column in just_dummies.columns:
    if column not in df.columns:
        df[column] = just_dummies[column]
df.head(2)

,Username,Repository Name,Description,Last Update Date,Language,Number of Stars,Tags,Url,Gravatar,assembly,...,rascal,ruby,rust,scala,shell,swift,tex,typescript,vim script,vue
0,freeCodeCamp,freeCodeCamp,The https://freeCodeCamp.com open source codeb...,2017-06-24T15:56:17Z,javascript,290k,"nonprofits,certification,curriculum,react,node...",http://github.com/freeCodeCamp/freeCodeCamp,https://avatars0.githubusercontent.com/u/98925...,0,...,0,0,0,0,0,0,0,0,0,0
1,twbs,bootstrap,"The most popular HTML, CSS, and JavaScript fra...",2017-06-24T15:40:21Z,javascript,112k,"javascript,css,html,bootstrap,jekyll-site,scss,",http://github.com/twbs/bootstrap,https://avatars0.githubusercontent.com/u/29185...,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for tag in tags:
    if tag not in df.columns:
        df[tag] = 0
    try:
        if len(tag) > 4 :
            df.loc[df['Repository Name'].str.contains(tag), tag] = 1
            df.loc[df['Description'].str.contains(tag), tag] = 1
        df.loc[df['Tags'].str.contains(tag + ","), tag] = 1
    except Exception:
        pass
# Remove columns not needed
df.set_index(['Repository Name'])
COLUMNS_TO_REMOVE_LIST = ['', 'Username', 'Repository Name', 'Description',
                          'Last Update Date', 'Language', 'Number of Stars', 'Tags', 'Url','Gravatar' ,'Unnamed: 0']
# Stop words: links to (https://github)
RAGE_TAGS_LIST = [ 'github','algorithms','learn','learning','http' ,'https']



for column in COLUMNS_TO_REMOVE_LIST + RAGE_TAGS_LIST:
    try:
        del df[column]
    except Exception:
        pass

df.columns = df.columns.str.lower()


print ("Our final label matrix for repo list is")
df.head(2)

In [ ]:
print("")